In [562]:
from starter import *
import pickle
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

## Load data

In [2]:
train = pd.read_csv('train.csv')
holidays = pd.read_csv('holidays.csv')
weather = pd.read_csv('weather.csv')
metadata = pd.read_csv('metadata.csv')
submission_format = pd.read_csv('submission_format.csv')
submission_frequency = pd.read_csv('submission_frequency.csv')

In [546]:
submission_frequency = pickle.load(open('submission_frequency.p', 'rb'))
metadata = pickle.load(open('metadata.p', 'rb'))
holidays = pickle.load(open('holidays.p', 'rb'))
weather_grouped = pickle.load(open('weather_grouped.p', 'rb'))
train = pickle.load(open('train.p', 'rb'))
submission_format = pickle.load(open('submission_format.p', 'rb'))
df = pickle.load(open('df.p', 'rb'))

## Feature Engineering

In [244]:
df['Holiday'].fillna(0, inplace=True)
df['isHoliday'].fillna(0, inplace=True)
df.drop(['Unnamed: 0_x','Unnamed: 0_y'], axis=1, inplace=True)

In [48]:
df.head()

,obs_id,SiteId,Timestamp,Value,Date,Time,Hour,DayofWeek,Year,Month,Temperature,Distance,Holiday,isHoliday
0,744519,1,2014-09-03,9.096555e+05,2014-09-03,00:00:00,0,Wednesday,2014,9,19.60,22.921092,0,0.0
1,7627564,1,2014-09-04,1.748273e+06,2014-09-04,00:00:00,0,Thursday,2014,9,21.30,22.921092,0,0.0
2,7034705,1,2014-09-05,0.000000e+00,2014-09-05,00:00:00,0,Friday,2014,9,23.35,22.921092,0,0.0
3,5995486,1,2014-09-06,0.000000e+00,2014-09-06,00:00:00,0,Saturday,2014,9,21.60,22.921092,0,0.0
4,7326510,1,2014-09-07,0.000000e+00,2014-09-07,00:00:00,0,Sunday,2014,9,15.80,22.921092,0,0.0


In [245]:
submission_frequency.head()

,ForecastId,ForecastPeriodNS,ForecastPeriodMin,Period_Quarter,Period_Hour,Period_Days
0,1,86400000000000,1440.0,0,0,1
1,2,86400000000000,1440.0,0,0,1
2,3,86400000000000,1440.0,0,0,1
3,4,86400000000000,1440.0,0,0,1
4,5,3600000000000,60.0,0,1,0


In [267]:
# align SiteId to ForecastId
id_align = submission_format.groupby(by = 'ForecastId').mean()
id_align['ForecastId'] = id_align.index
id_align.drop(['obs_id', 'Value'], axis=1, inplace=True)

In [269]:
submission_frequency = submission_frequency.merge(id_align, how='left', on='ForecastId')
site_type = submission_frequency.groupby(by='SiteId').mean()

In [497]:
metadata.head()

,SiteId,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,WednesdayIsDayOff,ThursdayIsDayOff,FridayIsDayOff,SaturdayIsDayOff,SundayIsDayOff,Temperature
SiteId,,,,,,,,,,,,
1,1,1387.205119,15.0,18.0,0,0,0,0,0,1,1,0
2,2,6098.278376,30.0,18.0,0,0,0,0,0,1,1,0
3,3,10556.293605,5.0,18.0,0,0,0,0,0,1,0,0
5,5,12541.181277,30.0,18.0,0,0,0,0,0,1,1,0
6,6,9150.195373,30.0,18.0,0,0,0,0,0,1,1,0


## Model for scoring

In [49]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [226]:
def RMSE(y, pred):
    return mean_squared_error(y, pred)**0.5

In [88]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [502]:
# acquire siteid data
def get_site_data(data=None, site_id=None, features=['Value','Temperature']):
    df = data[data.SiteId==site_id]
    df.index = df.Timestamp
    df = df[features]
    return df

In [505]:
# segment and interpolate data
def prepare_site(data=None, site_id=None, features= ['Value', 'Temperature'], method='linear'):
    df = get_site_data(data, site_id, features)
    try:
        # start where there isn't consecutive NaNs
        start_index = df[(df.Temperature.isnull()==False)&(df.Temperature.shift(-1).isnull()==False)].index[0]
        df = df.loc[start_index:,:]
        df['Temperature'].interpolate(method, inplace=True)
    except:
        print('No temperature data')
    return df

In [510]:
df2=prepare_site(df, 2)

In [458]:
def create_master_data(data=None, sites=None, features=['Value', 'Temperature'], method='linear'):
    master_df = pd.DataFrame()
    for site in sites:
        threshold_size = data[data.SiteId==site].shape[0]*0.5
        site_features = features.copy()
        if data[data.SiteId==site].Temperature.isnull().value_counts()[0] < threshold_size:
            site_features.remove('Temperature')
        df = prepare_site(data, site, site_features)
        print('Finished site {} with rows {}'.format(site, df.shape[0]))
        master_df = pd.concat([master_df, df], axis=0)
    print('Final rows: {}'.format(master_df.shape[0]))
    return master_df

In [683]:
# normalize and reframe features
def reframe_features(data=None, n_in=1):
    values = data.values
    # ensure all data is float
    values = values.astype('float32')
    # normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(values)
    # frame as supervised learning
    reframed = series_to_supervised(scaled, n_in, 1)
    reframed.drop(reframed.columns[[reframed.shape[1] - i for i in range(1,data.shape[1])]], axis=1, inplace=True)
    print(reframed.head())
    return reframed, scaler

In [512]:
reframed_data = reframe_features(df2, 2)

   var1(t-2)  var2(t-2)  var1(t-1)  var2(t-1)   var1(t)
2   0.201850   0.763602   0.222406   0.769231  0.215942
3   0.222406   0.769231   0.215942   0.769231  0.180349
4   0.215942   0.769231   0.180349   0.795497  0.167053
5   0.180349   0.795497   0.167053   0.769231  0.179651
6   0.167053   0.769231   0.179651   0.769231  0.157865


In [603]:
# split train/test sets from reframed data
def split_train_test_val(reframed):
    values = reframed.values
    n_train_days = int(len(values) * 0.7)
    train = values[:n_train_days, :]
    test = values[n_train_days:, :]
    # Split into input and outputs
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]
    # Reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    return train_X, train_y, test_X, test_y

In [604]:
train_X, train_y, test_X, test_y = split_train_test_val(reframed_data)

(24283, 1, 4) (24283,) (10407, 1, 4) (10407,)


In [713]:
# build multivariate LSTM model with validation
def create_LSTM_val(train_X=None, train_y=None, test_X=None, test_y=None, epochs=50, batch_size=100):
    model = Sequential()
    model.add(LSTM(4, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    history = model.fit(train_X, train_y, epochs=epochs,
                                batch_size=batch_size, validation_data=(test_X, test_y),
                                verbose=1, shuffle=False)
    return model

In [529]:
multi_model = create_LSTM_val(train_X, train_y, test_X, test_y, epochs=50)

Train on 24283 samples, validate on 10407 samples
Epoch 1/50
24283/24283 [==============================] - 3s 108us/step - loss: 0.0663 - val_loss: 0.0045
Epoch 2/50
24283/24283 [==============================] - 1s 25us/step - loss: 0.0201 - val_loss: 0.0032
Epoch 3/50
24283/24283 [==============================] - 1s 27us/step - loss: 0.0122 - val_loss: 0.0025
Epoch 4/50
24283/24283 [==============================] - 1s 24us/step - loss: 0.0086 - val_loss: 0.0026
Epoch 5/50
24283/24283 [==============================] - 1s 24us/step - loss: 0.0078 - val_loss: 0.0027
Epoch 6/50
24283/24283 [==============================] - 1s 23us/step - loss: 0.0074 - val_loss: 0.0026
Epoch 7/50
24283/24283 [==============================] - 1s 23us/step - loss: 0.0071 - val_loss: 0.0026
Epoch 8/50
24283/24283 [==============================] - 1s 23us/step - loss: 0.0068 - val_loss: 0.0026
Epoch 9/50
24283/24283 [==============================] - 1s 23us/step - loss: 0.0065 - val_loss: 0.0025
Epoc

In [517]:
yhat = multi_model.predict(test_X)

In [518]:
# compare forecast to actual
def forecast_score(pred, actual_input, actual_output):
    actual_input = actual_input.reshape((actual_input.shape[0], actual_input.shape[2]))
    
    # Invert scaling for forecast
    inv_pred = np.concatenate((pred, actual_input[:, 1:]), axis=1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(inv_pred)
    inv_pred = scaler.inverse_transform(inv_pred)
    inv_pred = inv_pred[:,0]
    
    # Invert scaling for actual
    actual_output = actual_output.reshape((len(actual_output), 1))
    inv_output = np.concatenate((actual_output, actual_input[:, 1:]), axis=1)
    inv_output = scaler.inverse_transform(inv_output)
    inv_output = inv_output[:,0]
    rmsle = RMSE(inv_output, inv_pred)
    
    return rmsle

In [519]:
forecast_score(yhat, test_X, test_y)

0.016920891150341268

## Model for site prediction

In [666]:
# concatenate train and test sets
def load_site(train=None, test=None, site_id=None, weather=weather_grouped, holidays=holidays):
    train = train[train.SiteId==site_id]
    test = test[test.SiteId==site_id]
    train.loc[:,'ForecastId'] = 0
    df = pd.concat([train, test], axis=0)
    df = df.merge(weather, how='left', on=['Timestamp', 'SiteId'])
    df = df.merge(holidays, how='left', on=['Date','SiteId'])
    df['Holiday'].fillna(0, inplace=True)
    df['isHoliday'].fillna(0, inplace=True)
    df.drop(['Unnamed: 0_x','Unnamed: 0_y'], axis=1, inplace=True)
    return df

In [701]:
df1 = load_site(train, submission_format,1)

In [702]:
# segment and interpolate data for site
def prepare_data(data=None, features= ['Value', 'Temperature'], method='linear'):
    data.index = data.Timestamp
    data = data[features]
    try:
        # start where there isn't consecutive NaNs
        start_index = data[(data.Temperature.isnull()==False)&(data.Temperature.shift(-1).isnull()==False)].index[0]
        data = data.loc[start_index:,:]
        data['Temperature'].interpolate(method, inplace=True)
    except:
        print('No temperature data')
    return data

In [703]:
df1 = prepare_data(df1, features=['Value','Temperature','isHoliday'])

In [731]:
n_train_days = train[train.SiteId==1].shape[0]-1
reframed_data, scaler = reframe_features(df1, 2)

   var1(t-2)  var2(t-2)  var3(t-2)  var1(t-1)  var2(t-1)  var3(t-1)   var1(t)
2   0.090837   0.825079        0.0   0.174579   0.860906        0.0  0.000000
3   0.174579   0.860906        0.0   0.000000   0.904110        0.0  0.000000
4   0.000000   0.904110        0.0   0.000000   0.867229        0.0  0.000000
5   0.000000   0.867229        0.0   0.000000   0.744995        0.0  0.196209
6   0.000000   0.744995        0.0   0.196209   0.733404        0.0  0.326227


In [732]:
train_X, train_y, test_X, test_y = split_train_test(reframed_data, n_train_days)

(899, 1, 6) (899,) (236, 1, 6) (236,)


In [733]:
n_train_days = train[train.SiteId==1].shape[0]-1
n_features = df1.shape[1]

In [734]:
# split train/test sets from reframed data
def split_train_test(reframed, n_train_days):
    values = reframed.values
    train = values[:n_train_days, :]
    test = values[n_train_days+1:, :]
    # Split into input and outputs
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]
    # Reshape input to be 3D [samples, timesteps, features]
    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    return train_X, train_y, test_X, test_y

In [740]:
# build multivariate LSTM model for predictions
def create_LSTM_pred(train_X=None, train_y=None, epochs=50, batch_size=100):
    model = Sequential()
    model.add(LSTM(4, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    history = model.fit(train_X, train_y, epochs=epochs,
                                batch_size=batch_size,
                                verbose=1, shuffle=False)
    return model

In [736]:
df1_model = create_LSTM_pred(train_X, train_y, epochs=50)

Epoch 1/50
899/899 [==============================] - 3s 3ms/step - loss: 0.3428
Epoch 2/50
899/899 [==============================] - 0s 36us/step - loss: 0.3138
Epoch 3/50
899/899 [==============================] - 0s 33us/step - loss: 0.2863
Epoch 4/50
899/899 [==============================] - 0s 26us/step - loss: 0.2604
Epoch 5/50
899/899 [==============================] - 0s 29us/step - loss: 0.2363
Epoch 6/50
899/899 [==============================] - 0s 28us/step - loss: 0.2140
Epoch 7/50
899/899 [==============================] - 0s 27us/step - loss: 0.1933
Epoch 8/50
899/899 [==============================] - 0s 30us/step - loss: 0.1744
Epoch 9/50
899/899 [==============================] - 0s 30us/step - loss: 0.1571
Epoch 10/50
899/899 [==============================] - 0s 28us/step - loss: 0.1414
Epoch 11/50
899/899 [==============================] - 0s 30us/step - loss: 0.1274
Epoch 12/50
899/899 [==============================] - 0s 27us/step - loss: 0.1148
Epoch 13/50
89

In [737]:
# make predictions with LSTM model
def model_predict(model, test_X, n_features):
    yhat = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    inv_yhat = np.concatenate([yhat, test_X], axis=1)[:,:n_features]
    inv_yhat = scaler.inverse_transform(inv_yhat)[:,0]
    return inv_yhat

In [738]:
inv_yhat = model_predict(df1_model, test_X, n_features)

In [ ]:
# add features from metadata to site data
def add_meta(data, site_id):
    